<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Approfondissements

Dans ce complément, nous allons aborder deux sujets tout à fait optionnels, et qui s'adressent surtout à ceux d'entre vous qui souhaitent compléter l'étude de python dans notre contexte.

Par ailleurs nos deux sujets sont assez disparates&nbsp;:

 * dans un premier temps nous allons comparer les performances obtenues entre python et C/C++,
 * ensuite nous étudierons à titre de curiosité une autre technique pour l'implémentation des tableaux à deux dimensions, basé sur un dictionnaire indexé par des tuples. Cette technique sera utilisée en semaine 5 pour l'algorithme UPGMA, mais le présent complément est à cet égard totalement optionnel : on rappelera l'essentiel avant de voir l'algorithme en question, mais ici nous voulons creuser un peu plus ces notions. 

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

***

## Considérations de performances

Le langage python, j'espère que vous en êtes à présent convaincus, très simple à mettre en oeuvre. En particulier la possibilité d'exécuter du code à la volée, comme nous l'avons fait dans ce MOOC, le rend très attractif comparé, notamment, aux langages compilés comme C et C++.

Cette facilité d'utilisation comporte par contre un coût non négligeable en termes de performances. Un des plus gros défauts de python est notamment d'être **relativement gourmand en mémoire**, et cela quel que soit le type d'application.

Dans le cas d'algorithmes comme celui de Needleman & Wunsch, où il s'agit de faire de manière très systématique des parcours de tableaux, on perd égalemement beaucoup **en termes de vitesse de calcul**, et cela est d'autant plus sensible que l'algorithme ici est quadratique - cela signifie que pour comparer deux chaines de taille $n$ on fait de l'ordre de $n^2$ opérations.

Je me suis par exemple livré à un benchmark pour comparer la vitesse qu'on obtient, avec le même algorithme, selon qu'on l'écrit en C++ ou en python. 

Les résultats sont assez étonnants. Vous trouverez - à titre indicatif - les sources de ces différents essais, ainsi que des données plus circonstanciées, dans le repository git ci-dessous
https://github.com/parmentelat/flotbioinfo/tree/master/benchmarks

Dans ce benchmark on a&nbsp;:
  * toujours utilisé python2 et `xrange` au lieu de `range` - comme on l'a fait jusqu'ici pour ne pas embrouiller les novices&nbsp;;
  * implémenté une optimisation qui consiste à parcourir exactement le rectangle, plutôt que de parcourir tout le triangle puis à éliminer les points qui ne sont pas dans le rectangle&nbsp;;
  * comparé les performances obtenues avec l'interpréteur usuel de python (Cython) et celles obtenues avec [pypy](http://pypy.org/) pour exécuter le code python tel quel.

En résumé toutefois, sur un test qui consiste à calculer la distance entre deux chaines constituées de 2000 `A` et 2000 `B` respectivement, on observe&nbsp;:
 * le code python avec l'interpréteur usuel est de l'ordre de **120 fois** plus lent que le code C++ ;
 * le code python interprété avec `pypy` est de l'ordre de **4 à 5 fois** plus lent que le code C++ ;
 * l'optimisation qui calcule précisement les bornes de l'intersection de  la diagonale avec le triangle fournit en gros un gain de $1/4$ à $1/6$ selon les cas ;
 * le code le plus rapide (C++) traite les deux mots de 2000 caractères en 36 ms, au lieu de 154 ms pour le code sous `pypy` et 4.28 s pour la version python standard.

##### Cohabitation C et python

Concluons cette section en signalant que la lenteur relative de python n'est toutefois pas rédhibitoire, car il est très pratique de pouvoir disposer d'un interpréteur pour essayer interactivement diverses approches - comme vous avez pu le constater vous-mêmes d'ailleurs.

Aussi dans la pratique on trouve très souvent des solutions mixtes où les algorithmes rapides sont implémentés en C ou C++ et *wrappés* en python, c'est-à-dire qu'on peut appeler le code rapide sur des données python.

******

## Double tableau&nbsp;: un dictionnaire indexé sur des tuples

Dans le notebook précédent, nous avons utilisé un tableau à double entrée implémenté comme une liste de listes.
Il existe une autre méthode pour modéliser un tableau à double entrée, qui est plus *pythonique* quoique sans doute un peu moins efficace, et qui consiste à utiliser un dictionnaire indexé sur des tuples. 

Sachez qu'on aura l'occasion d'utiliser cette technique la semaine prochaine (ou ce sera ré-expliqué plus simplement). Notre propos ici est de saisir cette occasion pour creuser un peu certains aspects de python.

Nous avons croisé ici et là la notion de tuple déjà, voici quelques mots pour préciser un peu ce concept.

##### Objets mutables et immutables

Pour rester simple, voyons pour commencer la notion d'objet **mutable**. Un objet en python est dit mutable lorsqu'on peut le modifier; par exemple, une **liste est mutable** dans le sens où on peut modifier **en place** un de ses éléments. Par exemple&nbsp;:

In [ ]:
# une liste est un objet mutable
list1 = [0, 1, 2]
# si une deuxième variable pointe vers la liste
list2 = list1
# et qu'on modifie la première liste
list1[1] = 100
# alors on a modifié les deux variables
print("list1=", list1, "list2=", list2)

A contrario, **une chaine** par exemple est un objet **immutable**. Une chaine ne peut pas être modifiée&nbsp;:

In [ ]:
# une chaine est immutable
chaine = "abc"
# si on essaie de changer un caractère on reçoit une exception
try:
    chaine[1] = 'Z'
except Exception as e:
    print("OOPS", e)

On pourrait, bien entendu, réaffecter la variable `chaine` à une autre chaine, mais ce n'est pas le même concept.

##### Une clé de dictionnaire doit être immutable

Remarquons pour commencer qu'un dictionnaire ne peut, pour des raisons d'implémentation, utiliser que des clés **immutables**&nbsp;:

In [ ]:
# un dictionnaire vide
d = {}

# on peut insérer une clé qui est une chaine, car la chaine est immutable
d['abc'] = 123

# mais on ne peut pas utiliser une clé qui est une liste
try:
    d[ [1, 2] ] = 123
except Exception as e:
    print("OOPS", e)
    
# à ce stade d a une seule clé 'abc'
print(d)

C'est ici qu'intervient le tuple. C'est une structure qui ressemble beaucoup à une liste, en ce sens qu'on peut y mettre une collection ordonnée d'objets. 

In [ ]:
# un tuple s'écrit avec des virgules - et, si on veut des parenthèses
t1 = (1, "abc")
print("t1", t1)

t2 = 2, "def"
print("t2", t2)

##### Un tuple peut servir de clé

Mais le tuple est un objet **immutable**, et du coup on peut s'en servir comme clé dans un dictionnaire&nbsp;:

In [ ]:
# un tableau à double entrée comme un dictionnaire de tuples
costs = {}
costs[ (100, 100) ] = 'abc'
costs[ (1000, 0) ] = [1, 2, 3]
print("le dictionnaire", costs)

In [ ]:
# on retrouve les données comme un dictionnaire normal
costs[ (100, 100)]

Il s'agit d'une structure très pratique, pour stocker des tableaux a multiples entrées, notamment dans les cas où c'est difficile de prévoir à l'avance la taille - typiquement lorsque le tableau a plein de trous.

##### Syntaxe des tuples

Signalons enfin que c'est **la virgule**, et non pas tant les parenthèses, qui font le tuple. Ainsi on peut écrire indifféremment&nbsp;:

In [ ]:
t1 = (1, 2)
t2 = 1, 2
t1 == t2

Si bien qu'on peut tout simplement écrire&nbsp;:

In [ ]:
costs[100, 100]

##### Exercice

À titre d'exercice, vous pouvez vous amuser à reprendre le code du complément précédent, pour utiliser un dictionnaire de tuples plutôt que la liste de listes que nous avons utilisée, et ensuite comparer les performances obtenues dans les deux cas. 

Pour ce dernier point on peut utiliser&nbsp;:
 * soit [le module `timeit`](https://docs.python.org/2/library/timeit.html) que nous avons déjà vu en Semaine 2, Séquence 2, pour la traduction de l'ADN en ARN, 
 * soit "à la main" avec [le module `time`](https://docs.python.org/2/library/time.html) et en particulier la fonction `time.time()`, comme ceci&nbsp;:

In [ ]:
# une illustration de la fonction time
import time

# un flottant en secondes qui correspond à l'heure de maintenant
l = []
beg = time.time()
for i in range(2000):
    sl = []
    for j in range(2000):
        sl.append(j)
    l.append(sl)
end = time.time()
print("le tout a duré {} en secondes", end-beg)